In [41]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
import os
import psycopg2
from config import *

In [43]:
# Connect to the database
conn = psycopg2.connect(
    host="ec2-54-147-36-107.compute-1.amazonaws.com",
    database="d1i9m18bh0nee",
    user="nrdsxccgistivu",
    password=pg_password)

# Read the data from the database
df1 = pd.read_sql_query('SELECT * FROM "eminem_comments"',con=conn)
df2 = pd.read_sql_query('SELECT * FROM "psy_comments"',con=conn)
df3 = pd.read_sql_query('SELECT * FROM "katyperry_comments"',con=conn)
df4 = pd.read_sql_query('SELECT * FROM "lmfao_comments"',con=conn)
df5 = pd.read_sql_query('SELECT * FROM "shakira_comments"',con=conn)


In [44]:
# concatenate the dataframes
df = pd.concat([df1, df2, df3, df4, df5])

df



,id,content,class
0,0,lovely girl talk xxx,1
1,1,always end coming back song br,0
2,2,sister received new rel nofollow class ot hash...,1
3,3,cool,0
4,4,hello palastine,1
...,...,...,...
362,365,love song sing camp time,0
363,366,love song two reasons africa born beautiful so...,0
364,367,wow,0
365,368,shakira u wiredo,0


In [31]:
# drop the comment_id, date, and video columns
#df = df.drop(['COMMENT_ID', 'DATE', 'video', 'AUTHOR'], axis=1)

#df

KeyError: "['COMMENT_ID' 'DATE' 'video' 'AUTHOR'] not found in axis"

In [45]:
# Use machine learning to create a model that can predict whether a comment is spam or not spam

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['content'], df['class'], random_state=0)

# Use CountVectorizer to create document-term matrices from X_train and X_test
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

# Use Naive Bayes to predict the star rating for X_test_dtm
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)

# Calculate the accuracy of your model
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

# Calculate the AUC
y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]
metrics.roc_auc_score(y_test, y_pred_prob)

# Print the confusion matrix
metrics.confusion_matrix(y_test, y_pred_class)



array([[199,  29],
       [ 12, 233]], dtype=int64)

In [34]:
# import the what happened csv
df_new = pd.read_csv('csv/what happened. comments.csv')

df_new

,id,author,thumbsup,text,# of replies,replies
0,UgxthbCBDyVQkjwuOEF4AaABAg,JC,0,You have done much worse things - take the int...,1,"[""Hey there troll, how ya doin'?""]"
1,Ugy6uVlNSOSWrJvm3R54AaABAg,NiceToSee You,0,I thought I was over the issue but then I real...,0,[]
2,Ugx3ujNO4ZpGV7T4HhJ4AaABAg,Sohu,0,wait - so he cheated? it sounds like the dude ...,1,"[""Look at the comments, it's explained multipl..."
3,UgzucTxFKtl3-47Z-8d4AaABAg,Heldermaior,0,What is the issue here? Did he force himself o...,1,"[""Look at the comments, it's explained multipl..."
4,Ugwt9nqZ-TApbLxEiBx4AaABAg,Raquel Nery,0,Oh gosh,0,[]
...,...,...,...,...,...,...
51287,UgxyP0ITZrA2CF7JEKB4AaABAg,Jacob lewis Griffiths,4,First,0,[]
51288,UgxoVr952wevxpc1hxR4AaABAg,Brandon Nguyen,13,NO WAY,0,[]
51289,UgzRybYMa-6mmIS7KxF4AaABAg,CR1Tr,9,Good,0,[]
51290,UgwufUpi1-5qHNLg2L94AaABAg,Boydism23,11,I’m early early,0,[]


In [35]:
# drop id author thumbsup # of replies and replies columns
df_new = df_new.drop(['id', 'author', 'thumbsup', '# of replies', 'replies'], axis=1)

In [36]:
df_new

,text
0,You have done much worse things - take the int...
1,I thought I was over the issue but then I real...
2,wait - so he cheated? it sounds like the dude ...
3,What is the issue here? Did he force himself o...
4,Oh gosh
...,...
51287,First
51288,NO WAY
51289,Good
51290,I’m early early


In [37]:
# drop na
df_new = df_new.dropna()

In [38]:
# use the model to predict the class of the new comments
X_new_dtm = vect.transform(df_new['text'])
new_pred_class = nb.predict(X_new_dtm)

# add the new class to the dataframe
df_new['class'] = new_pred_class

df_new

c:\Users\macdk\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,text,class
0,You have done much worse things - take the int...,1
1,I thought I was over the issue but then I real...,0
2,wait - so he cheated? it sounds like the dude ...,0
3,What is the issue here? Did he force himself o...,0
4,Oh gosh,0
...,...,...
51287,First,0
51288,NO WAY,0
51289,Good,0
51290,I’m early early,1


In [39]:
# % of comments marked spam
len(df_new[df_new['class'] == 1])/len(df_new)

0.5992707988067617